In [1]:
import os
import cv2
import utils
import numpy as np
from numpy import genfromtxt
from keras.models import load_model
from keras.utils import CustomObjectScope
import tensorflow as tf
with CustomObjectScope({'tf': tf}):
  model = load_model('model/nn4.small2.v1.h5')

Using TensorFlow backend.
/Users/macworld/opt/anaconda3/envs/py2/lib/python2.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
weights = utils.weights
weight_dict = utils.load_weights()

In [3]:
for name in weights:
    model.get_layer(name).set_weights(weight_dict[name])

In [4]:
import cryptography
from cryptography.fernet import Fernet
import base64
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

def keygeneration(mailid):
#getting and storing the key
    #with open("key.jpg", "rb") as img_file:
    key = ''
    password = base64.b64encode(str.encode(mailid))
    #filen = raw_input("enter the destination where the encoded key will be saved")
    #password_e=password.encode()#converting the password to byte form
    salt='salt'.encode()#creating a random 16 bits salt string
    #Password based key derivation function 2
    kdf = PBKDF2HMAC( #An instance of HashAlgorithm
        algorithm=hashes.SHA256(),
        length=32,#lenght of key in bytes
        salt=salt,#defining salt string
        iterations=100000,#number of iterations to perform hash function,highr the number,lower the chances of brute force
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password)) #derives one or more secret keys 
                                                        #In conjunction with non-secret parameters to derive 
                                                        #one or more keys from a common secret value
                                                        #can be used one time only
                                                        #Final key
    
    return key

#---------------ENCRYPTION--------------

def encryption(key):
    
    input_file= raw_input("Enter the path of the file location to be encrypted ")
    output_file= raw_input("Enter the destination of the encrypted text ")

    with open(input_file, 'rb') as f:
        data = f.read()   #reading the file to be encryted
    
    fernet = Fernet(key)   #creating object of fernet class with our key
    encrypted = fernet.encrypt(data) #encrypting

    with open(output_file, 'wb') as f: #writing the encryted msg
        f.write(encrypted)

    print("Encryption done.....!!!!!")
    
#---------------DECRYPTION---------------

def decryption(key):
    
    input_file_d = raw_input(" Enter the path of the file location to be decrypted ")
    output_file_d= raw_input("Enter the destination of the decrypted text ")

    with open(input_file_d, 'rb') as f:
        data = f.read()   #reading the file to be decryted
    
    fernet = Fernet(key)   #creating object of fernet class with our key
    encrypted = fernet.decrypt(data) #encrypting

    with open(output_file_d, 'wb') as f: #writing the decryted msg
        f.write(encrypted)
    
    print("Decryption done.....!!!!!")


### Converting image to embedding

In [5]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

### function to recognise image

In [113]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.65:
        return str(name)
    else:
        return None

### Capturing image samples

In [7]:
def sampling():
    mail = raw_input("Enter the mailid")
    mid = keygeneration(mail)
    cam = cv2.VideoCapture(0)

    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    count = 0
    while(True):
        ret, img = cam.read()
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
            count += 1
            # Save the captured image into the datasets folder
            cv2.imwrite("encryptedimages/"+mid + ".jpg", img[y1:y2,x1:x2])
            cv2.imshow('image', img)
        k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 10: # Take 30 face sample and stop video
             break
    cam.release()
    cv2.destroyAllWindows()

## This check if the gien creds is the person or not

In [8]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("encryptedimages/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    
    count = 0
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #while(count > 5):
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            print(identity)
            print("calling creds")
            try:
                decryption(identity)
            except:
                print("You are not who you are")
            if(count == 0):
                break
            '''
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)
        '''
        key = cv2.waitKey(100)
        if(count == 0):
            break
        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Lets Run the program

In [20]:
%%time
#capturing new facesperosn
sampling()

Enter the mailidperson10
CPU times: user 3.22 s, sys: 202 ms, total: 3.43 s
Wall time: 9.45 s


In [32]:
%%time
#getting the key for encryption
mailid = raw_input("Enter the senders mail id")
key = keygeneration(mailid)
#encrypting
encryption(key)

Enter the senders mail idperson10
Enter the path of the file location to be encrypted msg.txt
Enter the destination of the encrypted text encrypt10.txt
Encryption done.....!!!!!
CPU times: user 88.7 ms, sys: 4.73 ms, total: 93.4 ms
Wall time: 14.6 s


In [115]:
%%time
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

Euclidean distance from KV3uRuUPlkWI0RTUhh2tW6XSV2yCGXZ8hwc2heqAsRM= is 0.62633884
Euclidean distance from J3w73197PcyU-hgaSC8P2rfUVjav2_s9sEeHL7WMKUI= is 0.6377438
Euclidean distance from TeY41yIOQXtu6bl0saBCTF3Myl8z0B03SyDNban3hk8= is 0.7594459
Euclidean distance from 32Nif1E_3VBwBt0O9s6AgFzxULNCjSRmOYTJI2L7Eok= is 1.1760316
Euclidean distance from O9eDqJOUL1lA3O8nrfOhc8WAKcG3zAoO4O6JNU36_5M= is 0.8534429
Euclidean distance from rittika is 0.761748
Euclidean distance from HXBFuVV4keX978GDZpcAYeiRqXf_T2mF1QaPa1WC53I= is 0.6816282
Euclidean distance from rohitsinha123 is 1.15056
Euclidean distance from qoFDhPJopRuJo-uEyk_5KaihppWfYxkfJex7MnuOs5o= is 0.62391996
Euclidean distance from b-zkUfL5eDkVPKk8WTCU1Wrf5rqugoSbVLfLCyPq0QM= is 0.683758
Euclidean distance from -XQGU8wZzUMgjZDh4aI_-i6P_K4TqcsDgJEu0mAuhyU= is 0.80852747
Euclidean distance from ebFQIKAFBR9NihKWwyd6fH-o1FYpbcqfJ4eXtzaMzd0= is 0.81742024
qoFDhPJopRuJo-uEyk_5KaihppWfYxkfJex7MnuOs5o=
calling creds
 Enter the path of the fi